In [1]:
import pandas as pd
from collections import Counter 
from helper import load_data,pipeline, unique_drugs, generate_drugs_df
import re

%pylab inline 
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
df_original = load_data()
#c = unique_drugs(trials)
#df_data, exceptions = generate_drugs_df(c, trials)
df_original = df_original.reset_index()


## expanded unique drugs to get a dataframe we can group by
dupes = []
for idx, item_list in enumerate(df_original["Primary Drugs"]):
    split_list = item_list.split(",")
    if len(split_list)>1:
        for i in split_list:
            df_original.append(df_original.iloc[idx,:])
            df_original.iloc[-1,-3]=i
        dupes.append(idx)

df_trials = df_original.drop(dupes)


In [3]:
drug_list = df_trials["Primary Drugs"].unique()
phase_of_trial_list = df_trials["Phase of Trial"].unique()
df_num_of_pariticpants = pd.DataFrame(np.zeros([len(drug_list),len(phase_of_trial_list)]),columns=phase_of_trial_list)
df_num_of_pariticpants['drug'] = drug_list
df_num_of_pariticpants = df_num_of_pariticpants.set_index('drug')
grouped_phases = df_trials.groupby(["Primary Drugs","Phase of Trial"])["Planned Subject Number"].agg('sum')
cleaner_groups = grouped_phases.reset_index()

for drug in drug_list:
    for item in cleaner_groups[cleaner_groups["Primary Drugs"]==drug].iterrows():   
        phase = list(item)[1]['Phase of Trial']
        count = list(item)[1]['Planned Subject Number']
        df_num_of_pariticpants.loc[drug,phase] = count

    
    

In [4]:
df_drugs = pd.read_excel('bcdrugs.xlsx')
df_drugs['drug'] = df_drugs.apply(lambda x: x["Drug Name"].split("-")[0], axis=1)
df_num_of_pariticpants_red = df_num_of_pariticpants.drop('Phase 0',axis=1)
df_num_of_pariticpants_red = df_num_of_pariticpants_red[(df_num_of_pariticpants_red['Phase I']!=0) | (df_num_of_pariticpants_red['Phase II']!=0) | (df_num_of_pariticpants_red['Phase I/II']!=0)]

In [5]:
# Create boolean to determine if the current passed. Based on whether there were participants from the next phase
#phases = [item for item in df_num_of_pariticpants_red.columns if 'I' in item]
df_num_of_pariticpants_red['Phase I Pass'] = df_num_of_pariticpants_red['Phase II']!=0
df_num_of_pariticpants_red['Phase II Pass'] = df_num_of_pariticpants_red['Phase III']!=0
df_num_of_pariticpants_red['Phase I/II Pass'] = df_num_of_pariticpants_red['Phase III']!=0
df_num_of_pariticpants_red['Phase III Pass'] = df_num_of_pariticpants_red['Phase IV']!=0
df_num_of_pariticpants_red['Phase II/III Pass'] = df_num_of_pariticpants_red['Phase IV']!=0

In [31]:
df_data = df_num_of_pariticpants_red

Getting number of organizations for each drug

In [6]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [7]:
def remove_org_dupes(x):
    drop_list = []
    #print(type(x))
    if type(x) != type('astring'):
        return []
    if x == np.nan:
        return []
    split_orgs = x.split(",")
    if len(split_orgs)>1:
        for idx1, i in enumerate(split_orgs[:-1]):
            for idx2, j in enumerate(split_orgs[idx1+1:]):
                #print(i,idx1, j ,idx2, fuzz.ratio(i,j))
                if fuzz.ratio(i,j) > 50:
                    drop_list.append(idx1+idx2+1)
        #print(f'Drop list: {drop_list}')
        #print(f'FINAL: {split_orgs[list(set(drop_list))]}')
        #split_orgs[not set(drop_list)F
        #print (list(set(drop_list)))
        output = np.delete(split_orgs, list(set(drop_list)))
        return [item.strip() for item in output]
    return [item.strip() for item in [x]]

In [8]:
#trials["Organisations"] = trials["Organisations"].apply(str)
df_trials['clean Organisations'] = df_trials['Organisations'].apply(remove_org_dupes)

In [9]:
df_trials['number of clean Organisations'] = df_trials['clean Organisations'].apply(lambda x:len(x))

In [11]:
df_org_count = df_trials[['Primary Drugs','number of clean Organisations']]
df_org_count["Primary Drugs"] = df_org_count["Primary Drugs"].apply(lambda x: x.strip())

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [21]:
grouped_orgs = df_org_count.groupby("Primary Drugs")["number of clean Organisations"].agg('max')
grouped_orgs = grouped_orgs.reset_index()

In [32]:
df_data = df_data.merge(grouped_orgs, left_index=True, right_on="Primary Drugs", how='left')
df_data.set_index('Primary Drugs', inplace=True)